In [5]:
import pandas as pd
import numpy as np
# data = pd.read_csv('/kaggle/input/new-tip-smote/Dcomsmote.csv')
# data = data.iloc[:, :-1]
# Paths and data loading (adjust paths accordingly)
path1 = '/kaggle/input/ace-dataset/features/'
AAC = pd.read_csv(path1 + 'ACE_AAC.csv').iloc[:, 1:]
ASDC = pd.read_csv(path1 + 'ACE_ASDC.csv').iloc[:, 1:]
OPF_7bit_type_1 = pd.read_csv(path1 + 'opf_7bit_type_1_features.csv').iloc[1:, 1:]
OPF_7bit_type_2 = pd.read_csv(path1 + 'opf_7bit_type_2_features.csv').iloc[1:, 1:]
OPF_7bit_type_3 = pd.read_csv(path1 + 'opf_7bit_type_3_features.csv').iloc[1:, 1:]
OPF_10bit = pd.read_csv(path1 + 'opf_10bit_features.csv').iloc[1:, 1:]
esmv1 = pd.read_csv(path1 + 'esmv1_feat_ACE.csv').iloc[:, :]
esm2 = pd.read_csv(path1 + 'esm2_t6_8M_feat_ACE.csv').iloc[:, :]
prot_t5 = pd.read_csv(path1 + 'ACE_embeddings_prot_t5_xl_bfd.csv').iloc[1:, 1:]

# Concatenate all features into a single array
data = np.column_stack((AAC, ASDC, OPF_7bit_type_1, OPF_7bit_type_2, OPF_7bit_type_3, OPF_10bit, esmv1, esm2, prot_t5))

data_ = pd.DataFrame(data)
#R='R100'
data = data_

In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

class MaskingNoiseAutoencoder(tf.keras.Model):
    def __init__(self, n_input, n_hidden, dropout_probability=0.2):
        super(MaskingNoiseAutoencoder, self).__init__()
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.dropout_probability = dropout_probability

        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(self.n_input,)),
            tf.keras.layers.Dropout(rate=self.dropout_probability),
            tf.keras.layers.Dense(self.n_hidden, activation=tf.nn.softplus)
        ])

        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(self.n_input, activation=tf.nn.softplus)
        ])

        self.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss=tf.keras.losses.MeanSquaredError())

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

    def partial_fit(self, X):
        cost = self.train_on_batch(X, X)
        return cost

    def calc_total_cost(self, X):
        cost = self.evaluate(X, X, verbose=0)
        return cost

    def transform(self, X):
        encoded = self.encoder.predict(X)
        return encoded

    def generate(self, hidden=None):
        if hidden is None:
            hidden = np.random.normal(size=(1, self.n_hidden))
        generated = self.decoder(tf.constant(hidden, dtype=tf.float32))
        return generated.numpy()

    def reconstruct(self, X):
        return self.predict(X)

    def getWeights(self):
        return self.encoder.get_weights()[0]

    def getBiases(self):
        return self.encoder.get_weights()[1]


data_numeric = data.apply(pd.to_numeric, errors='coerce')


data_numeric.dropna(inplace=True)

# Preprocess data
X = np.array(data_numeric.iloc[:, :])  # Assuming the features are in columns 1 onwards
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_val = train_test_split(X_scaled, test_size=0.2, random_state=42)


n_hidden_list = [50,100,150,200,250,300,350,400,450,500]  

for n_hidden in n_hidden_list:
    print(f"Training Autoencoder with {n_hidden} hidden units")
    
    # Define parameters
    autoencoder = MaskingNoiseAutoencoder(
        n_input=X_scaled.shape[1],    # Number of features
        n_hidden=n_hidden,
        dropout_probability=0.2
    )

    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train Autoencoder
    autoencoder.fit(
        X_train, X_train,
        epochs=1000,
        batch_size=16,
        validation_data=(X_val, X_val),
        callbacks=[early_stopping]
    )

    
    features = autoencoder.transform(X_scaled)

    # Save extracted features to CSV
    output_file = f'AEDN-{n_hidden}.csv'
    features_df = pd.DataFrame(data=features, index=data_numeric.index)

    
    new_column_names = [f'AEDN-{n_hidden}_{i}' for i in range(features_df.shape[1])]
    features_df.columns = new_column_names

    
    features_df.to_csv(output_file, index=True, header=True)
    print(f"Features saved to {output_file}")


2024-07-03 09:05:14.057534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 09:05:14.057642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 09:05:14.180410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Training Autoencoder with 50 hidden units
Epoch 1/1000
33/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1161 

I0000 00:00:1719997528.229863     152 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1719997528.245123     152 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0955

W0000 00:00:1719997529.181704     150 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


...

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0656

W0000 00:00:1719997659.999431     150 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1719997660.321516     150 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0651 - val_loss: 0.0230
Epoch 2/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0219 - val_loss: 0.0188
Epoch 3/1000


W0000 00:00:1719997660.546893     152 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0176 - val_loss: 0.0159
Epoch 4/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0146 - val_loss: 0.0136
Epoch 5/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0124 - val_loss: 0.0118
Epoch 6/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0108 - val_loss: 0.0104
Epoch 7/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0093 - val_loss: 0.0095
Epoch 8/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0082 - val_loss: 0.0088
Epoch 9/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0078 - val_loss: 0.0083
Epoch 10/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0071 - val_loss: 0.0079
Epoch 11/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0063 - val_loss: 0.0076
Epoch 12/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0060 - val_loss: 0.0072
Epoch 13/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0056 - val_loss: 0.0070
Epoch 14/1000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.